In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma, Pinecone
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.chains.summarize import load_summarize_chain
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import pandas as pd
import numpy as np
from config import OPENAI_API_KEY, PINECONE_API_KEY, PINECONE_API_ENV
import pinecone

In [7]:
loader = DirectoryLoader('../data/eur/', glob='*.txt')
data = loader.load()
print (f'You have {len(data)} document(s) in your data')

You have 3 document(s) in your data


In [8]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts  = text_splitter.split_documents(data)
print (f'Now you have {len(texts)} documents')

Now you have 87 documents


In [13]:
texts

[Document(page_content='Про затвердження Порядку заповнення та видачі митницею сертифіката з перевезення (походження) товару EUR.1 або EUR-MED\n\nВідповідно до статті 46 глави 7 розділу II, пункту 14 частини другої статті 544 глави 74 розділу XX Митного кодексу України, підпункту 5 пункту 4 Положення про Міністерство фінансів України, затвердженого постановою Кабінету Міністрів України від 20 серпня 2014 року N 375, з метою вдосконалення та прискорення процедури заповнення та видачі митницею сертифіката з перевезення (походження) товару EUR.1 або EUR-MED згідно з положеннями Регіональної конвенції про пан-євро-середземноморські преференційні правила походження, а також міжнародних угод про вільну торгівлю, укладених у встановленому законодавством порядку, якими передбачено процедуру заповнення та видачі митницею сертифіката з перевезення (походження) товару EUR.1 або EUR-MED,\n\nНАКАЗУЮ:', metadata={'source': '..\\data\\eur\\file1.txt'}),
 Document(page_content='НАКАЗУЮ:\n\n1. Затверди

In [9]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [15]:
# initialize pinecone
vectordb = pinecone.init(
api_key=PINECONE_API_KEY,  # find at app.pinecone.io
environment=PINECONE_API_ENV  # next to api key in console
)   

In [16]:
index_name = "eurtest" # put in the name of your pinecone index here

In [17]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)
docsearch